## Equity Market Data Analysis Reports

In [1]:
import configparser
from pyspark.sql import SparkSession
from equity_market_data_analysis import Tracker

In [2]:
spark = SparkSession.builder.getOrCreate()

config = configparser.ConfigParser()
config.read('../config_local.ini')

output_container = config.get('APP_CONFIG', 'OutputContainer')
eod_report_dir = config.get('APP_CONFIG', 'EodReportDir')
analytical_report_dir = config.get('APP_CONFIG', 'AnalyticalReportDir')
processing_date = config.get('PRODUCTION', 'ProcessingDate')   

tracker = Tracker('', config) # name not needed for report-only


21/07/25 15:21:16 WARN Utils: Your hostname, pySpark-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
21/07/25 15:21:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/07/25 15:21:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### EOD reports

In [3]:
# quotes

spark.read.parquet(output_container + '/' + eod_report_dir + '/quote_dt=' + processing_date)\
.show(20)

21/07/25 15:21:19 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-azure-file-system.properties,hadoop-metrics2.properties


+----------+------+--------+--------------------+------------+-------------------+------+--------+------+--------+
|  trade_dt|symbol|exchange|            event_tm|event_seq_nb|         arrival_tm|bid_pr|bid_size|ask_pr|ask_size|
+----------+------+--------+--------------------+------------+-------------------+------+--------+------+--------+
|2020-08-06|  SYMA|  NASDAQ|2020-08-06 10:22:...|           7|2020-08-06 09:30:00| 77.76|     100| 79.25|     100|
|2020-08-06|  SYMA|  NASDAQ|2020-08-06 10:29:...|           8|2020-08-06 09:30:00| 75.60|     100| 76.96|     100|
|2020-08-06|  SYMA|  NASDAQ|2020-08-06 11:15:...|          14|2020-08-06 09:30:00| 79.20|     100| 80.12|     100|
|2020-08-06|  SYMA|  NASDAQ|2020-08-06 11:23:...|          15|2020-08-06 09:30:00| 74.84|     100| 76.31|     100|
|2020-08-06|  SYMA|  NASDAQ|2020-08-06 11:49:...|          18|2020-08-06 09:30:00| 75.37|     100| 76.57|     100|
|2020-08-06|  SYMA|  NASDAQ|2020-08-06 11:55:...|          19|2020-08-06 09:30:0

In [4]:
# trades

spark.read.parquet(output_container + '/' + eod_report_dir + '/trade_dt=' + processing_date)\
.show(20)

+----------+------+--------+--------------------+------------+-------------------+--------+
|  trade_dt|symbol|exchange|            event_tm|event_seq_nb|         arrival_tm|trade_pr|
+----------+------+--------+--------------------+------------+-------------------+--------+
|2020-08-06|  SYMA|  NASDAQ|2020-08-06 10:42:...|          10|2020-08-06 09:30:00|   78.93|
|2020-08-06|  SYMA|  NASDAQ|2020-08-06 13:09:...|          30|2020-08-06 09:30:00|   78.31|
|2020-08-06|  SYMA|  NASDAQ|2020-08-06 19:09:...|          80|2020-08-06 09:30:00|   78.62|
|2020-08-06|  SYMA|    NYSE|2020-08-06 12:00:...|          20|2020-08-06 09:30:00|   76.16|
|2020-08-06|  SYMA|    NYSE|2020-08-06 16:58:...|          60|2020-08-06 09:30:00|   78.32|
|2020-08-06|  SYMA|    NYSE|2020-08-06 20:49:...|          90|2020-08-06 09:30:00|   75.71|
|2020-08-06|  SYMB|  NASDAQ|2020-08-06 12:01:...|          20|2020-08-06 09:30:00|   32.47|
|2020-08-06|  SYMB|  NASDAQ|2020-08-06 14:15:...|          40|2020-08-06 09:30:0

### Analytical report

In [5]:
# analytical quote 

spark.read.parquet(output_container + '/' + analytical_report_dir + '/quote_dt=' + processing_date)\
.show(20)

+----------+------+--------+--------------------+--------------+----------------------+--------------------+
|  trade_dt|symbol|exchange|            event_tm|prior_trade_pr|prior_mov_avg_trade_pr|prior_close_trade_pr|
+----------+------+--------+--------------------+--------------+----------------------+--------------------+
|2020-08-06|  SYMC|  NASDAQ|2020-08-06 19:04:...|        159.07|                159.07|              158.02|
|2020-08-06|  SYMC|  NASDAQ|2020-08-06 19:12:...|        159.07|                159.07|              158.02|
|2020-08-06|  SYMC|  NASDAQ|2020-08-06 19:20:...|        159.07|                159.07|              158.02|
|2020-08-06|  SYMC|  NASDAQ|2020-08-06 19:28:...|        159.07|                159.07|              158.02|
|2020-08-06|  SYMC|  NASDAQ|2020-08-06 19:32:...|        159.07|                159.07|              158.02|
|2020-08-06|  SYMC|  NASDAQ|2020-08-06 19:41:...|        159.07|                159.07|              158.02|
|2020-08-06|  SYMC|

### Show status of jobs

In [6]:
tracker.get_job_status('ingestion_' + processing_date)

('ingestion_2020-08-06',
 datetime.datetime(2021, 7, 25, 14, 39, 31, 841710),
 'success')

In [7]:
tracker.get_job_status('reports_' + processing_date)

('reports_2020-08-06',
 datetime.datetime(2021, 7, 25, 14, 40, 54, 824857),
 'success')